# Customer Demography Dashboard

**Muhammad Satrio Pinoto Negoro**

**Linkedin:** https://www.linkedin.com/in/satriopino/

Import Dependencies

In [2]:
import pandas as pd
from nomiden import reader as nr
import numpy as np
import plotly.express as px

## Read Customer Data

In [4]:
customer_data = pd.read_csv('customer_all.csv')
customer_data.head()

,CustomerID,NIK,Annual_Income,Spending_Score,Profession,Work_Experience,Family_Size
0,1,5203172209040014,15000000,39,Healthcare,1,4
1,2,1705032211020016,35000000,81,Engineer,3,3
2,3,7606055105030001,86000000,6,Engineer,1,1
3,4,7317074601000015,59000000,77,Lawyer,0,2
4,5,3215114911920008,38000000,40,Entertainment,2,6


Checking Data (Data type, row data, null data)

In [5]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CustomerID       1107 non-null   int64 
 1   NIK              1107 non-null   int64 
 2   Annual_Income    1107 non-null   int64 
 3   Spending_Score   1107 non-null   int64 
 4   Profession       1107 non-null   object
 5   Work_Experience  1107 non-null   int64 
 6   Family_Size      1107 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 60.7+ KB


**Correct the data type of each data column to the data type it should be**

> **Category Column**
- Profession

> **Float64 Column**
- Annual_Income
- Work_Experience

> **Int64 Column**
- Family_Size
- Spending_Score

> **Object Column**
- CustomerID
- NIK

In [6]:
customer_data.columns

Index(['CustomerID', 'NIK', 'Annual_Income', 'Spending_Score', 'Profession',
       'Work_Experience', 'Family_Size'],
      dtype='object')

In [7]:
customer_data[['CustomerID', 'NIK']] = customer_data[['CustomerID', 'NIK']].astype('object')
customer_data[['Annual_Income', 'Work_Experience']] = customer_data[['Annual_Income', 'Work_Experience']].astype('float64')
customer_data['Profession'] = customer_data['Profession'].astype('category')

In [8]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CustomerID       1107 non-null   object  
 1   NIK              1107 non-null   object  
 2   Annual_Income    1107 non-null   float64 
 3   Spending_Score   1107 non-null   int64   
 4   Profession       1107 non-null   category
 5   Work_Experience  1107 non-null   float64 
 6   Family_Size      1107 non-null   int64   
dtypes: category(1), float64(2), int64(2), object(2)
memory usage: 53.5+ KB


## Data Pre-processing and Feature Engineering

### Data Enrichment

Data enrichment adalah proses menambah dan memperbaiki data mentah (raw data) supaya kualitas data yang dimiliki menjadi lebih baik dan komplet. Kita dapat melengkapi informasi dalam data kita menggunakan nomor NIK. Untuk dapat mengekstraksi informasi dari nomor NIK, kita dapat menggunakan library `nomiden`. Dokumentasi lengkap: https://nomiden.readthedocs.io/

Untuk mendapatkan seluruh informasi dari nomor NIK dalam suatu kolom, kita dapat menggunakan sintaks berikut.

`df['NIK'].apply(lambda x: nr.NIK(x).<ATRIBUT>)`

Beberapa atribut yang dapat kita gunakan:
- `.province`: Provinsi
- `.city`: Kota/Kabupaten
- `.district`: Kecamatan
- `.gender`: Gender
- `.birthdtm`: Tanggal lahir dalam datetime
- `.birthday`: Tanggal lahir dalam string
- `.age`: Usia
- `.nth_person`: Kode registrasi dari Dukcapil
- `.all_info`: Seluruh informasi

In [11]:
nik_generate = pd.DataFrame(customer_data['NIK'].apply(lambda x: nr.NIK(x).all_info).tolist())
nik_generate.head()

,NIK,province,city,district,gender,birth_datetime,birthday,age,regist_code
0,5203172209040014,NUSA TENGGARA BARAT,KAB. LOMBOK TIMUR,Labuhan Haji,Male,2004-09-22,22 September 2004,19,14
1,1705032211020016,BENGKULU,KAB. SELUMA,Talo,Male,2002-11-22,22 November 2002,21,16
2,7606055105030001,SULAWESI BARAT,KAB. MAMUJU TENGAH,Karossa,Female,2003-05-11,11 May 2003,20,1
3,7317074601000015,SULAWESI SELATAN,KAB. LUWU,Belopa,Female,2000-01-06,06 January 2000,23,15
4,3215114911920008,JAWA BARAT,KAB. KARAWANG,Cibuaya,Female,1992-11-09,09 November 1992,31,8


In [12]:
customer_data.head()

,CustomerID,NIK,Annual_Income,Spending_Score,Profession,Work_Experience,Family_Size
0,1,5203172209040014,15000000.0,39,Healthcare,1.0,4
1,2,1705032211020016,35000000.0,81,Engineer,3.0,3
2,3,7606055105030001,86000000.0,6,Engineer,1.0,1
3,4,7317074601000015,59000000.0,77,Lawyer,0.0,2
4,5,3215114911920008,38000000.0,40,Entertainment,2.0,6


**Merge DataFrame (Customer Data & Customer NIK Data)**

In [13]:
customer_merge = customer_data.merge(right=nik_generate, on='NIK')
customer_merge.head()

,CustomerID,NIK,Annual_Income,Spending_Score,Profession,Work_Experience,Family_Size,province,city,district,gender,birth_datetime,birthday,age,regist_code
0,1,5203172209040014,15000000.0,39,Healthcare,1.0,4,NUSA TENGGARA BARAT,KAB. LOMBOK TIMUR,Labuhan Haji,Male,2004-09-22,22 September 2004,19,14
1,2,1705032211020016,35000000.0,81,Engineer,3.0,3,BENGKULU,KAB. SELUMA,Talo,Male,2002-11-22,22 November 2002,21,16
2,3,7606055105030001,86000000.0,6,Engineer,1.0,1,SULAWESI BARAT,KAB. MAMUJU TENGAH,Karossa,Female,2003-05-11,11 May 2003,20,1
3,4,7317074601000015,59000000.0,77,Lawyer,0.0,2,SULAWESI SELATAN,KAB. LUWU,Belopa,Female,2000-01-06,06 January 2000,23,15
4,5,3215114911920008,38000000.0,40,Entertainment,2.0,6,JAWA BARAT,KAB. KARAWANG,Cibuaya,Female,1992-11-09,09 November 1992,31,8


In [14]:
customer_merge['birth_year'] = customer_merge['birth_datetime'].dt.year

**Categorization Data (Binning)** 

In [15]:
gen_bins =  [0, 1900, 1927, 1945, 1964, 1980, 1996, 2012, np.inf]
gen_names = ['Lost', 'Greatest', 'Silent', 'Boomers', 'Gen. X', 'Gen. Y (Millenials)', 'Gen. Z (Zoomers)', 'Alpha']

# simpan ke kolom 'generation'
customer_merge['generation'] = pd.cut(x=customer_merge['birth_year'], bins=gen_bins, labels=gen_names)
customer_merge.head()

,CustomerID,NIK,Annual_Income,Spending_Score,Profession,Work_Experience,Family_Size,province,city,district,gender,birth_datetime,birthday,age,regist_code,birth_year,generation
0,1,5203172209040014,15000000.0,39,Healthcare,1.0,4,NUSA TENGGARA BARAT,KAB. LOMBOK TIMUR,Labuhan Haji,Male,2004-09-22,22 September 2004,19,14,2004,Gen. Z (Zoomers)
1,2,1705032211020016,35000000.0,81,Engineer,3.0,3,BENGKULU,KAB. SELUMA,Talo,Male,2002-11-22,22 November 2002,21,16,2002,Gen. Z (Zoomers)
2,3,7606055105030001,86000000.0,6,Engineer,1.0,1,SULAWESI BARAT,KAB. MAMUJU TENGAH,Karossa,Female,2003-05-11,11 May 2003,20,1,2003,Gen. Z (Zoomers)
3,4,7317074601000015,59000000.0,77,Lawyer,0.0,2,SULAWESI SELATAN,KAB. LUWU,Belopa,Female,2000-01-06,06 January 2000,23,15,2000,Gen. Z (Zoomers)
4,5,3215114911920008,38000000.0,40,Entertainment,2.0,6,JAWA BARAT,KAB. KARAWANG,Cibuaya,Female,1992-11-09,09 November 1992,31,8,1992,Gen. Y (Millenials)


In [17]:
customer_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CustomerID       1107 non-null   object        
 1   NIK              1107 non-null   object        
 2   Annual_Income    1107 non-null   float64       
 3   Spending_Score   1107 non-null   int64         
 4   Profession       1107 non-null   category      
 5   Work_Experience  1107 non-null   float64       
 6   Family_Size      1107 non-null   int64         
 7   province         1107 non-null   object        
 8   city             1107 non-null   object        
 9   district         1107 non-null   object        
 10  gender           1107 non-null   object        
 11  birth_datetime   1107 non-null   datetime64[ns]
 12  birthday         1107 non-null   object        
 13  age              1107 non-null   int64         
 14  regist_code      1107 non-null   int64  

In [18]:
customer_merge[['province', 'city', 'district', 'gender']] = customer_merge[['province', 'city', 'district', 'gender']].astype('category')

In [19]:
customer_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CustomerID       1107 non-null   object        
 1   NIK              1107 non-null   object        
 2   Annual_Income    1107 non-null   float64       
 3   Spending_Score   1107 non-null   int64         
 4   Profession       1107 non-null   category      
 5   Work_Experience  1107 non-null   float64       
 6   Family_Size      1107 non-null   int64         
 7   province         1107 non-null   category      
 8   city             1107 non-null   category      
 9   district         1107 non-null   category      
 10  gender           1107 non-null   category      
 11  birth_datetime   1107 non-null   datetime64[ns]
 12  birthday         1107 non-null   object        
 13  age              1107 non-null   int64         
 14  regist_code      1107 non-null   int64  

**Data Exporting**

In [20]:
customer_merge.to_csv('customer_merge.csv', index=False)

In [21]:
customer_merge.to_pickle('customer_merge.pkl')

## Exploratory Data Analysis

Exploratory Data Analysis (**EDA**) mengacu pada proses melakukan investigasi awal pada data, seringkali dengan tujuan untuk lebih mengenal karakteristik data. EDA dilakukan dengan bantuan ringkasan statistik dan visualisasi sederhana untuk melihat struktur data yang kita miliki.

EDA:
- Melihat struktur/bentuk data
- Melihat statistika data
- Memvisualisasikan data

In [22]:
customer_merge.shape

(1107, 17)

In [23]:
customer_merge.describe()

,Annual_Income,Spending_Score,Work_Experience,Family_Size,birth_datetime,age,regist_code,birth_year
count,1.107000e+03,1107.00000,1107.000000,1107.000000,1107,1107.000000,1107.000000,1107.000000
mean,1.062517e+08,50.64047,3.967480,3.645890,1981-05-25 04:27:58.048780480,42.028004,9.887082,1980.897922
min,0.000000e+00,1.00000,0.000000,1.000000,1955-01-15 00:00:00,16.000000,1.000000,1955.000000
25%,6.982750e+07,27.00000,1.000000,2.000000,1968-03-02 12:00:00,29.000000,5.000000,1968.000000
50%,1.024130e+08,50.00000,3.000000,3.000000,1982-07-09 00:00:00,41.000000,10.000000,1982.000000
75%,1.465350e+08,74.00000,7.000000,5.000000,1993-12-19 12:00:00,55.000000,15.000000,1993.000000
max,1.899450e+08,100.00000,17.000000,9.000000,2006-12-28 00:00:00,68.000000,19.000000,2006.000000
std,4.780587e+07,27.86101,3.904931,1.937353,NaN,15.168914,5.543587,15.167881


✏️ **Insight** : 
- The average customer work experience is 3.967480 ~ 4 years
- The average customer age is 42 years
- The youngest customer is 16 years old and the oldest customer is 68 years old
- The average customer spending score is 50.64047 ~ 51

In [24]:
customer_merge.describe(include='category')

,Profession,province,city,district,gender,generation
count,1107,1107,1107,1107,1107,1107
unique,10,34,414,1024,2,4
top,Artist,JAWA TIMUR,KAB. GARUT,Gondang,Female,Gen. Y (Millenials)
freq,355,108,12,3,669,359


✏️ **Insight** : 
- It turns out that most of the customers are **Women**
- Most customers are from **generation Y (Millennials)**
- Most customers are **artists**
- Most customers are from **East Java Province**

**See the distribution of customers from each "Province"**

**Customer count per province**

In [140]:
# Convert 'province' column to ordered categorical type
customer_merge['province'] = customer_merge['province'].astype('category')

# Filter based on the income range
cust_prov = customer_merge[customer_merge['Annual_Income'].between(left=30000000.0, right=100000000.0)]

# Create a crosstab
customer_per_province = pd.crosstab(index=cust_prov['province'],
                                     columns='Total Customers',
                                     colnames=[None]).sort_values(by='Total Customers', ascending=False)
customer_per_province

,Total Customers
province,
JAWA TIMUR,54
SUMATERA UTARA,43
JAWA BARAT,42
JAWA TENGAH,33
PAPUA,30
SUMATERA SELATAN,25
SULAWESI SELATAN,21
ACEH,19
NUSA TENGGARA TIMUR,17


In [72]:
customer_per_province = pd.crosstab(index=customer_merge['province'],
                   columns='Total Customers',
                   colnames=[None]).sort_values(by='Total Customers')
customer_per_province.sort_values(by='Total Customers', ascending=False).head(10)

,Total Customers
province,
JAWA TIMUR,108
JAWA BARAT,92
SUMATERA UTARA,85
JAWA TENGAH,85
PAPUA,78
NUSA TENGGARA TIMUR,59
SUMATERA SELATAN,46
ACEH,46
SULAWESI SELATAN,44


In [107]:
customer_per_province_plot = px.bar(customer_per_province.reset_index().tail(10), x='Total Customers', y='province', 
             title='Customer Count per Province', labels={'index': 'Province', 'province': 'Customer Count'},
             color_discrete_sequence=['black'])

# Customize the layout if needed
customer_per_province_plot.update_layout(
    xaxis_title='Customer Count',
    yaxis_title='Province',
    width=800,
    height=500
)

# Show the plot
customer_per_province_plot

**See the distribution of customers gender from each "Profession"**

In [108]:
customer_gender_prof_plot = px.bar(customer_merge.groupby(['Profession', 'gender']).size().reset_index(name='count').sort_values(by='count'),
             x='count', y='Profession', color='gender', barmode='group',
             title='Customer Gender Distribution per Profession',
             labels={'count': 'Customer Count'},
             category_orders={'gender': ['Male', 'Female']},
             color_discrete_map={'Male': 'black', 'Female': 'gray'})

# Customize the layout if needed
customer_gender_prof_plot.update_layout(
    xaxis_title='Customer Count',
    yaxis_title='Profession',
    width=800,
    height=500,
)

# Show the plot
customer_gender_prof_plot

C:\Users\pinos\AppData\Local\Temp\ipykernel_13936\3704270262.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\pinos\anaconda3\envs\nik\lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [113]:
customer_per_profession = pd.crosstab(index=customer_merge['Profession'],
                   columns='Total Customers',
                   colnames=[None])
customer_per_profession.sort_values(by='Total Customers', ascending=False)

,Total Customers
Profession,
Artist,355
Healthcare,202
Entertainment,114
Engineer,93
Doctor,84
Executive,83
Lawyer,77
Marketing,46
Homemaker,36


In [143]:
customer_age = customer_merge[customer_merge['age'].between(left=16, right=50)]
cust_prov = customer_age[customer_age['province'] == 'ACEH']
cust_prov.groupby(['Profession', 'gender']).size().reset_index(name='count').sort_values(by='count')

C:\Users\pinos\AppData\Local\Temp\ipykernel_13936\51424161.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Profession,gender,count
19,Not Employed,Male,0
16,Marketing,Female,0
14,Lawyer,Female,0
4,Engineer,Female,0
5,Engineer,Male,0
13,Homemaker,Male,0
12,Homemaker,Female,0
8,Executive,Female,0
18,Not Employed,Female,0
17,Marketing,Male,1


**Spending Score per Province**

In [60]:
customer_merge.groupby('province')['Spending_Score'].mean().sort_values(ascending=False).head(10)

C:\Users\pinos\AppData\Local\Temp\ipykernel_13936\2010101883.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



province
BALI                         79.800000
DKI JAKARTA                  63.200000
GORONTALO                    62.800000
MALUKU                       62.727273
KEPULAUAN BANGKA BELITUNG    61.200000
BANTEN                       60.695652
KALIMANTAN SELATAN           59.882353
SULAWESI TENGGARA            59.564103
SULAWESI UTARA               56.964286
KEPULAUAN RIAU               56.176471
Name: Spending_Score, dtype: float64

In [127]:
average_spending_per_generation = customer_merge.groupby('province')['Spending_Score'].mean().sort_values().reset_index().tail(10)

# Bar chart for average Spending Score per generation using Plotly
spending_per_province_plot = px.bar(average_spending_per_generation, x='Spending_Score', y='province', title='Average Spending Score per Province',
             color_discrete_sequence=['green'])

# Customize the layout if needed
spending_per_province_plot.update_layout(
    xaxis_title='Average Spending Score',
    yaxis_title='Province',
    width=800,
    height=500
)

# Show the plot
spending_per_province_plot

C:\Users\pinos\AppData\Local\Temp\ipykernel_13936\3373362053.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Customer Count per Generation**

In [119]:
customer_per_generation_plot = px.bar(customer_merge['generation'].value_counts().reset_index().sort_values(by='count'), x='count', y='generation', 
             title='Customer Count per Generation', labels={'index': 'Generation', 'generation': 'Customer Count'},
             color_discrete_sequence=['black'])  # Set bar color to black

# Customize the layout if needed
customer_per_generation_plot.update_layout(
    xaxis_title='Generation',
    yaxis_title='Customer Count',
    width=800,
    height=500,
)

# Show the plot
customer_per_generation_plot

**Customers Count across Indonesia**

In [128]:
# read data coordinate
coord = pd.read_csv("coordinate.csv")
coord.head()

,province,latitude,longitude
0,ACEH,4.36855,97.02530
1,SUMATERA UTARA,2.19235,99.38122
2,SUMATERA BARAT,-1.34225,100.07610
3,RIAU,0.50041,101.54758
4,JAMBI,-1.61157,102.77970


In [132]:
prov_gender = pd.crosstab(index=customer_merge['province'],
                   columns=customer_merge['gender'],
                   colnames=[None])
prov_gender['Total'] = prov_gender['Female'] + prov_gender['Male']
prov_gender.head(10)

,Female,Male,Total
province,,,
ACEH,26,20,46
BALI,3,2,5
BANTEN,16,7,23
BENGKULU,9,13,22
DAERAH ISTIMEWA YOGYAKARTA,11,6,17
DKI JAKARTA,2,3,5
GORONTALO,8,7,15
JAMBI,14,13,27
JAWA BARAT,65,27,92


In [133]:
df_map = prov_gender.merge(right=coord, on='province')
df_map.head()

,province,Female,Male,Total,latitude,longitude
0,ACEH,26,20,46,4.36855,97.02530
1,BALI,3,2,5,-8.23566,115.12239
2,BANTEN,16,7,23,-6.44538,106.13756
3,BENGKULU,9,13,22,-3.51868,102.53598
4,DAERAH ISTIMEWA YOGYAKARTA,11,6,17,-7.79560,110.36950


In [135]:
# scatter_mapbox
plot_map = px.scatter_mapbox(data_frame=df_map, lat='latitude', lon='longitude',
                             mapbox_style='carto-positron', zoom=3,
                             size='Total',
                             hover_name='province',
                             hover_data={'Male': True,
                                         'Female': True,
                                         'latitude': False,
                                         'longitude': False},
                            color_discrete_sequence=['green'])

plot_map

**Analysis Correlation**

Correlation between **Annual Income** and **Spending Score**

In [120]:
correlation_income_spending = customer_merge['Annual_Income'].corr(customer_merge['Spending_Score'])
print(f"Correlation Coefficient: {correlation_income_spending}")

Correlation Coefficient: 0.01651697122297926


✏️ **Insight** : 
> There is **no corelation** between **Customers Annual Income** and **Customers Spending Score**

Correlation between **Working Experiences** and **Spending Score**

In [123]:
correlation_experiences_spending = customer_merge['Work_Experience'].corr(customer_merge['Spending_Score'])
print(f"Correlation Coefficient: {correlation_experiences_spending}")

Correlation Coefficient: -0.0034983075194192567


✏️ **Insight** : 
> There is **no corelation** between **Work Experiences** and **Customers Spending Score**

Correlation between **Age** and **Spending Score**

In [124]:
correlation_age_spending = customer_merge['age'].corr(customer_merge['Spending_Score'])
print(f"Correlation Coefficient: {correlation_age_spending}")

Correlation Coefficient: -0.013471537864203255


✏️ **Insight** : 
> There is **no corelation** between **Age** and **Customers Spending Score**

Correlation between **Family Size** and **Spending Score**

In [125]:
correlation_familysize_spending = customer_merge['Family_Size'].corr(customer_merge['Spending_Score'])
print(f"Correlation Coefficient: {correlation_familysize_spending}")

Correlation Coefficient: -0.012109859385127769


✏️ **Insight** : 
> There is **no corelation** between **Family Size** and **Customers Spending Score**